In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
csv_file = "reduced_scaled_list_of_molecs.csv"
data = pd.read_csv(csv_file, header = 0)
print(data.head())

   Unnamed: 0                                 MolecularStructure         0  \
0           0  COc1ccccc1CC(NC(C)=O)C(=O)NC1CCN(c2nnnn2-c2ccc...  1.645728   
1           1             O=C(CSc1nc2cccnc2n1Cc1ccccc1)NCc1ccco1  0.378596   
2           2  Cc1cccc2cc(C[NH+](CC3CCCO3)C(c3nnnn3Cc3ccco3)C...  1.493713   
3           3                  CCN(CC)c1ccc2c(Cl)c(Br)c(=O)oc2c1 -0.419793   
4           4  CS(=O)(=O)N1CCc2cc(-c3csc(NC(=O)Cc4cccs4)n3)ccc21  0.848317   

          1         2         3         4         5         6         7  ...  \
0  0.780982  0.144738  0.320349  0.581260 -0.031954 -0.005198  0.103418  ...   
1  0.855898 -0.186397  0.483713 -0.122298 -0.053900  0.276698 -0.037492  ...   
2  0.932209  0.331717  1.115455  0.367590 -0.562439 -0.297631 -0.023151  ...   
3 -0.333148 -0.065550 -0.020718 -0.454826  0.390388 -0.226066 -0.155734  ...   
4  0.360801 -0.272237  0.470996 -0.528439  0.396609 -0.264172 -0.078749  ...   

         92        93        94        95        9

In [3]:
# Verwijderen van irrelevante kolommen
columns_to_drop = ["MolecularStructure"]  # Voeg hier de namen van de irrelevante kolommen toe
data = data.drop(columns=columns_to_drop)

data = data.dropna() #verwijderd rijen met ontbrekende waarden
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data["ALDH1_inhibition"] = label_encoder.fit_transform(data["ALDH1_inhibition"]) #yes = 1 no = 0
print(data.head())

   Unnamed: 0         0         1         2         3         4         5  \
0           0  1.645728  0.780982  0.144738  0.320349  0.581260 -0.031954   
1           1  0.378596  0.855898 -0.186397  0.483713 -0.122298 -0.053900   
2           2  1.493713  0.932209  0.331717  1.115455  0.367590 -0.562439   
3           3 -0.419793 -0.333148 -0.065550 -0.020718 -0.454826  0.390388   
4           4  0.848317  0.360801 -0.272237  0.470996 -0.528439  0.396609   

          6         7         8  ...        92        93        94        95  \
0 -0.005198  0.103418  0.201387  ... -0.045048 -0.016376  0.026453 -0.008133   
1  0.276698 -0.037492 -0.164586  ...  0.016992 -0.025591  0.008975  0.009656   
2 -0.297631 -0.023151  0.046421  ...  0.049565  0.043207 -0.133211  0.105803   
3 -0.226066 -0.155734 -0.361352  ... -0.031907 -0.021265 -0.001733 -0.005007   
4 -0.264172 -0.078749 -0.152650  ...  0.040359  0.031169 -0.009435 -0.038523   

         96        97        98        99       100  ALD

In [4]:

X = data.drop(columns=["ALDH1_inhibition"])
y = data["ALDH1_inhibition"]

# 20% testset, 80% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Schalen van de functies
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# Importeren van het machine learning-model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluatie van het model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Uitvoer van de evaluatieresultaten
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

#Score per molecule
y_pred_proba = model.predict_proba(X)
prediction = y_pred_proba
prediction = pd.DataFrame(prediction)
prediction.sort_values(by=[1], ascending=False, inplace=True)
prediction 

Accuracy: 0.9949367088607595
Precision: 0.9917355371900827
Recall: 0.9917355371900827
F1 Score: 0.9917355371900827


c:\Users\20202181\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


,0,1
617,0.28,0.72
1729,0.28,0.72
1853,0.29,0.71
208,0.30,0.70
327,0.30,0.70
...,...,...
1090,0.60,0.40
1080,0.60,0.40
1038,0.60,0.40
1,0.85,0.15
